In [43]:
from itertools import product
import sys
from PIL import Image
from os import listdir

import numpy as np
import torch
import torchvision
from torchvision import transforms, datasets
from train_models import test, Net
import lucent.optvis.render as render

In [23]:
model = Net()
model.load_state_dict(torch.load("mnist_cnn.pt"))

<All keys matched successfully>

In [16]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)
dataset1 = datasets.MNIST("./data", train=True, transform=transform)
dataset2 = datasets.MNIST("./data", train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1)
test_loader = torch.utils.data.DataLoader(dataset2)
test(model, device, test_loader)


Test set: Average loss: 0.0259, Accuracy: 9912/10000 (99%)



In [17]:
model

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [37]:
dataset1

5

In [41]:
model(dataset1[1][0][None, :])

tensor([[-2.0266e-06, -1.8269e+01, -1.3123e+01, -2.4371e+01, -2.2181e+01,
         -2.7081e+01, -1.8067e+01, -1.9923e+01, -1.8885e+01, -2.0621e+01]],
       grad_fn=<LogSoftmaxBackward0>)

In [44]:
layers = [
    "conv1",
    "conv2",
]


def get_all_layers(model, layers, X):
    hooks = [render.ModuleHook(getattr(model, layer)) for layer in layers]
    model(X)
    for hook in hooks:
        hook.close()
    return [hook.features for hook in hooks]

In [60]:
testing = torch.cat([dataset1[i][0][None, :] for i in range(len(dataset1))], 0)
testing.shape

torch.Size([60000, 1, 28, 28])

In [45]:
acts = get_all_layers(model, layers, dataset1[1][0][None, :])

In [47]:
for t in acts:
    print(t.shape)

torch.Size([1, 32, 26, 26])
torch.Size([1, 64, 24, 24])


In [48]:
torch.save(acts, "test_acts")

In [50]:
235 * 60000 / (1000 * 1000)

14.1

In [61]:
128 * 60000 / (100 * 1000)

76.8